In [1]:
# 데이터 프레임 관련
import pandas as pd

# XML 관련
import bs4
from os import name
from lxml import html
import xml.etree.ElementTree as et
from urllib.parse import urlencode, quote_plus, unquote

# API 호출 관련
import json
import requests

# 기타
from tqdm import tqdm
pd.set_option('display.width',1000)

In [ ]:
## 각 컬럼 값 (참고 : https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15057267)
"""
resultCode : 결과코드
resultMsg : 결과메시지
Deal Amount : 거래금액
Building Area : 건물 전용/연면적
Building Use : 건물주용도
Build Year : 건축년도
Classification of Share Dealing : 지분거래 구분
Deal Year : 계약년도
Plottage : 대지권면적
Dong : 물건 소재 법정동
Sigungu : 물건 소재 시군구
Land Use : 물건 소재 토지 용도지역
Deal Month : 계약월
Building Type : 일반/집합 건물 구분
Deal Day : 계약일
Regional Code : 지역코드
Floor : 거래물건 층(집합건물)
""" 

1. 현재 데이터 수집

In [3]:
# API 호출
url        = 'http://openapi.molit.go.kr/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcNrgTrade'
serviceKey = 'C94PKnigawlH2iISVWz1Itw9tVzyuOCbOvKgzCrGOUZWvs3XqrkhH+/ntypJ7kK2P2g16vLYCJif3TNfO1i1KA=='
params = {
      'serviceKey' : serviceKey
    , 'LAWD_CD'  : '11110'  # 구 단위
    , 'DEAL_YMD' : '202207' # 날짜
}
response = requests.get(url, params=params)
content  = response.text

# item 태그 분리
xml_obj = bs4.BeautifulSoup(content,'lxml-xml')
rows    = xml_obj.findAll('item')

# ver 4 (한 줄씩 데이터 프레임으로 만들어서 concat)
# 리스트에 값 담기
row_list   = [] # 행 값
name_list  = [] # 열이름 값
value_list = [] # 데이터 값
DF = pd.DataFrame() # 최종 값

# 데이터 수집
for i in tqdm(range(0, len(rows))):
    columns = rows[i].find_all()
    
    for j in range(0,len(columns)):
        name_list.append(columns[j].name)
        value_list.append(columns[j].text)
    
    row_list = value_list
    df_each  = pd.DataFrame([row_list], columns=name_list)
    DF = pd.concat([DF, df_each])
    
    # 초기화
    name_list  = []
    value_list = []

# CSV 파일로 저장 (추후 DB 연동 예정)
DF.to_excel('국토교통부_상업업무용 부동산 매매 신고 자료(202207).xlsx', encoding='utf-8')

100%|████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 1231.24it/s]


과거 데이터 수집

In [ ]:
# 진행중